In [4]:
import pandas as pd
import re
import string
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold,train_test_split
from tqdm import tqdm
import numpy as np
import time

answer = 'gak tau saya bu'
ReferenceAnswer = "5 Dampak Negatif Perkembangan Teknologi:\nKetergantungan Berlebihan: Orang-orang menjadi terlalu bergantung pada teknologi dan kehilangan kemampuan untuk menyelesaikan masalah sendiri.\nKecanduan Gadget: Penggunaan gadget yang berlebihan dapat menyebabkan kecanduan dan mengganggu kesehatan mental dan fisik.\nKesenjangan Digital: Tidak semua orang memiliki akses yang sama terhadap teknologi, sehingga terjadi kesenjangan antara yang kaya dan yang miskin.\nKejahatan Siber: Teknologi dapat digunakan untuk melakukan kejahatan, seperti penipuan, pencurian data, dan penyebaran berita bohong.\nPenyalahgunaan Informasi: Informasi yang beredar di internet tidak selalu akurat dan dapat disalahgunakan untuk menyebarkan kebencian dan propaganda."

# Dataset Creation (Tokenizer)
class EssayDataTokenizer(Dataset):
    def __init__(self, answers, students, tokenizer, max_length):
        self.answers = answers
        self.students = students
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.answers)

    def __getitem__(self, idx):
        answer = self.answers[idx]
        student = self.students[idx]

        answer_encoding = self.tokenizer.encode_plus(
            answer,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        student_encoding = self.tokenizer.encode_plus(
            student,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'answer_text' : answer,
            'answer_input_ids': answer_encoding['input_ids'].flatten(),
            'answer_attention_mask': answer_encoding['attention_mask'].flatten(),
            'student_text' : student,
            'student_input_ids': student_encoding['input_ids'].flatten(),
            'student_attention_mask': student_encoding['attention_mask'].flatten(),
        }

class IndoBERTForSTS(nn.Module):
    def __init__(self, bert_model):
        super(IndoBERTForSTS, self).__init__()
        self.bert = bert_model

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        print('pooler',len(outputs))
        return outputs[0]

# Function to calculate cosine similarity using PyTorch operations
def cosine_sim(a, b):
    cos_sim = nn.CosineSimilarity(dim=1)
    return cos_sim(a, b)

# function to test the model
def test_model(model, data_loader,device):
    with torch.no_grad():
        for d in data_loader:
            print('ini d',d)
            answer_input_ids = d["answer_input_ids"].to(device)
            answer_attention_mask = d["answer_attention_mask"].to(device)
            student_input_ids = d["student_input_ids"].to(device)
            student_attention_mask = d["student_attention_mask"].to(device)

            answer_outputs = model(
                input_ids=answer_input_ids,
                attention_mask=answer_attention_mask
            )

            student_outputs = model(
                input_ids=student_input_ids,
                attention_mask=student_attention_mask
            )

            cosine_scores = cosine_sim(answer_outputs, student_outputs)

    return cosine_scores

def runExperiment(tokenizer, bert_model):
    # create essay dataset of test
    test_dataset = EssayDataTokenizer(
        [ReferenceAnswer],
        [answer],
        tokenizer,
        512
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=True
    )

    model = IndoBERTForSTS(bert_model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('device used :',device)
    model.to(device)

    test_result = test_model(model, test_data_loader,device)

    return test_result

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
bert_model = BertModel.from_pretrained('indobenchmark/indobert-base-p2')

# model creation and assignment
model = IndoBERTForSTS(bert_model)

result = runExperiment(tokenizer=tokenizer, bert_model=model)
print('result', result)


device used : cpu
ini d {'answer_text': ['5 Dampak Negatif Perkembangan Teknologi:\nKetergantungan Berlebihan: Orang-orang menjadi terlalu bergantung pada teknologi dan kehilangan kemampuan untuk menyelesaikan masalah sendiri.\nKecanduan Gadget: Penggunaan gadget yang berlebihan dapat menyebabkan kecanduan dan mengganggu kesehatan mental dan fisik.\nKesenjangan Digital: Tidak semua orang memiliki akses yang sama terhadap teknologi, sehingga terjadi kesenjangan antara yang kaya dan yang miskin.\nKejahatan Siber: Teknologi dapat digunakan untuk melakukan kejahatan, seperti penipuan, pencurian data, dan penyebaran berita bohong.\nPenyalahgunaan Informasi: Informasi yang beredar di internet tidak selalu akurat dan dapat disalahgunakan untuk menyebarkan kebencian dan propaganda.'], 'answer_input_ids': tensor([[    2,   418,  3345,  3778,  1657,  1429, 30472, 10971,  4799, 30472,
           232, 30469,   232,   234,  1386,  6990,   126,  1429,    41,  3337,
          1352,    90,  3426,   80

In [2]:
import re
import string
import torch
from torch import clamp
from transformers import BertTokenizer,BertModel,BertConfig
from sklearn.metrics.pairwise import cosine_similarity

class TokenSimilarity:

    def load_pretrained(self,model):
        self.tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
        self.model = BertModel.from_pretrained('indobenchmark/indobert-base-p2')
        
    def __cleaning(self, text:str):
        # Replace punctuations with space
        text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)

        # Clear multiple spaces
        text = re.sub(r'\s+', ' ', text).strip()

        # Replace newlines with space
        text = text.replace("\n", " ")

        return text.lower()
        
    def __process(self, first_token:str, second_token:str):
        inputs = self.tokenizer([first_token, second_token],
                                max_length=self.max_length,
                                truncation=self.truncation,
                                padding=self.padding,
                                return_tensors='pt')

        print('inputs',inputs)
        attention = inputs.attention_mask

        outputs = self.model(**inputs)

        # get the weights from the last layer as embeddings
        embeddings = outputs.last_hidden_state# when used in older transformers version
        # embeddings = outputs.last_hidden_state # when used in newer one

        # add more dimension then expand tensor
        # to match embeddings shape by duplicating its values by rows
        mask = attention.unsqueeze(-1).expand(embeddings.shape).float()

        masked_embeddings = embeddings * mask
        
        # MEAN POOLING FOR 2ND DIMENSION
        # first, get sums by 2nd dimension
        # second, get counts of 2nd dimension
        # third, calculate the mean, i.e. sums/counts
        summed = masked_embeddings.sum(1)
        counts = clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed/counts

        # return mean pooling as numpy array
        return mean_pooled.detach().numpy()
        
    def predict(self, first_token:str, second_token:str,
                return_as_embeddings:bool=False, max_length:int=512,
                truncation:bool=True, padding:str="max_length"):
        self.max_length = max_length
        self.truncation = truncation
        self.padding = padding

        first_token = self.__cleaning(first_token)
        second_token = self.__cleaning(second_token)

        print(first_token, second_token)

        mean_pooled_arr = self.__process(first_token, second_token)
        if return_as_embeddings:
            return mean_pooled_arr

        # calculate similarity
        similarity = cosine_similarity([mean_pooled_arr[0]], [mean_pooled_arr[1]])

        return similarity
    
# Load the state dictionary
state_dict = torch.load('savedModels/savedModel-IndoBERT-base-exEpoch20-fold5.pth',map_location=torch.device('cpu'))

# Adjust the state dictionary keys
new_state_dict = {}
for key, value in state_dict.items():
    new_key = key.replace("bert.", "")
    new_state_dict[new_key] = value

# Define the configuration for your BERT model
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p2', num_labels=2)

model = BertModel(config)

# Load the state dictionary into the model
model.load_state_dict(new_state_dict)

modelPred = TokenSimilarity()
modelPred.load_pretrained(model)

token1 = """1. LDR (Light Dependent Resistor): Fungsi: Sensor cahaya yang mengubah intensitas cahaya menjadi nilai resistansi. Sensor: Fotodioda yang sensitif terhadap cahaya. Aplikasi: Lampu otomatis, alarm pencuri, sensor kamera. 2. Infrared (Inframerah): Fungsi: Mendeteksi objek berdasarkan radiasi panas yang dipancarkan. Sensor: Fotodioda atau foto transistor yang sensitif terhadap cahaya inframerah. Aplikasi: Remote control, sensor suhu, sensor gerakan. 3. Ultrasonic: Fungsi: Mengukur jarak dengan menggunakan gelombang suara ultrasonik. Sensor: Transduser yang menghasilkan dan menerima gelombang suara ultrasonik. Aplikasi: Sensor parkir, sensor level air, sensor robot."""

# token2 = """
# Storage Device : Hardware yang dapat menyimpan informasi dan data, baik itu dalam perangkat komputer maupun di luar\nInput Device : Perangkat keras komputer yang berfungsi untuk melakukan pemasukan data\nOutput Device : yang berfungsi untuk melakukan pengeluaran data
# """

token2 = """
Sapi atau Lembu adalah hewan ternak anggota famili Bovidae dan subfamili Bovinae. Sapi dipelihara terutama untuk dimanfaatkan susu dan dagingnya sebagai pangan manusia. Hasil sampingannya seperti kulit, jeroan, tanduk, dan kotorannya juga dimanfaatkan untuk berbagai keperluan manusia. Sektor Peternakan di kabupaten Bintan dalam beberapa tahun terakhir menunjukan peningkatan populasi yang sangat menggembirakan, untuk ternak sapi, jumlah populasi pada tahun 2016 tercatat 725 ekor, jumlah ini meningkat menjadi 978 ekor pada tahun 2019 dan menjadi 1.078 ekor pada tahun 2021. Jumlah peningkatan populasi ternak sapi ini, salah satunya dikarenakan Kabupaten Bintan melalui Kementrian Pertanian telah mencanangkan Program Upsus Siwab (Upaya Khusus Sapi indukan Wajib Bunting). Sektor peternakan di Kabupaten Bintan walaupun bukan menjadi sektor utama dalam program pembangunan yang dilaksanakan pemerintah daerah, namun merupakan faktor yang sama sekali tidak bisa dipandang sebelah mata karena sektor peternakan memberikan kontribusi yang sangat besar bagi indeks peningkatan daya beli masyarakat dan perekonomian masyarakat. Ternak sapi potong merupakan salah satu ternak yang banyak dibudidayakan dan diusahakan petani di Kabupaten Bintan. Ternak tersebut berperan sebagai sumber pendapatan, membuka kesempatan kerja dan sumber protein hewani. Populasi ternak sapi potong yang tinggi menunjukkan salah satu potensi dan peluang yang dapat dimanfaatkan untuk memberikan nilai tambah dalam usaha ternak sapi, meningkatkan konsumsi gizi keluarga akan protein hewani bahkan sebagai komoditas agribisnis. Peluang pengembangan sapi potong cukup besar, hal ini dipengaruhi oleh beberapa faktor antara lain tersedianya sapi bakalan dalam jumlah besar dan mutu yang relatif baik, tersedianya pakan ternak dalam jumlah cukup, tersedianya lahan pangonan, relatif mudahnya akses pemasaran, keterampilan petani yang memadai, sosial budaya yang menunjang dan adanya dukungan baik dari pihak swasta atau pemerintah.
"""


tokenArr = [token2]

for idx in range(len(tokenArr)):
  print('----Number ',idx+1,'-----')
  resultBERT = modelPred.predict(token1,tokenArr[idx])
  print('rb',resultBERT)

C:\Users\adima\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


----Number  1 -----
1 ldr light dependent resistor fungsi sensor cahaya yang mengubah intensitas cahaya menjadi nilai resistansi sensor fotodioda yang sensitif terhadap cahaya aplikasi lampu otomatis alarm pencuri sensor kamera 2 infrared inframerah fungsi mendeteksi objek berdasarkan radiasi panas yang dipancarkan sensor fotodioda atau foto transistor yang sensitif terhadap cahaya inframerah aplikasi remote control sensor suhu sensor gerakan 3 ultrasonic fungsi mengukur jarak dengan menggunakan gelombang suara ultrasonik sensor transduser yang menghasilkan dan menerima gelombang suara ultrasonik aplikasi sensor parkir sensor level air sensor robot sapi atau lembu adalah hewan ternak anggota famili bovidae dan subfamili bovinae sapi dipelihara terutama untuk dimanfaatkan susu dan dagingnya sebagai pangan manusia hasil sampingannya seperti kulit jeroan tanduk dan kotorannya juga dimanfaatkan untuk berbagai keperluan manusia sektor peternakan di kabupaten bintan dalam beberapa tahun tera